In [1]:
import numpy as np
import pandas as p
import matplotlib.pyplot as plt
import cPickle
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
import sklearn.model_selection as sms
documents_categories = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/documents_categories.csv")
#clicks_test = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/clicks_test.csv", dtype={"display_id": int, "ad_id": int}))
#documents_meta = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/documents_meta.csv")
#documents_entities = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/documents_entities.csv")
promoted_content = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/promoted_content.csv", dtype = {"ad_id":int, "document_id":int, "campaign_id":int, "advertiser_id":int})
#sample_submission = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/sample_submission.csv")
documents_topics = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/documents_topics.csv")
clicks_train = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/clicks_train.csv", dtype={"display_id": int, "ad_id": int, "clicked" : bool})
events = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/events.csv", dtype={"display_id": int, "uuid": str, "document_id" : int, "timestamp" : int, "platform" : str, "geo_location" : str})
#page_views = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/page_views.csv")
#page_views_sample = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/page_views_sample.csv")

In [2]:
#returns 2 tables, document on ad - doc-ad normalized score ; ad strength - ad normalized score
#requires: base table of train (or validation similiar to train) with document id of display id merged (from events)
def ad_on_doc_str(train):
    train_ad_count_per_display = train.groupby(['display_id'])['display_id'].agg({'ads_on_doc' : 'count'}).reset_index()
    train_temp = train.merge(train_ad_count_per_display, how = 'left', on = 'display_id')
    del train_ad_count_per_display
    document_on_ad = train_temp.groupby(['document_id','ad_id','ads_on_doc']).clicked.agg({'clicked' : 'sum', 'total' : 'count'}).reset_index()
    document_on_ad['uni_chance'] = 1 / document_on_ad['ads_on_doc']
    document_on_ad['clicked_percent'] = document_on_ad['clicked'] / document_on_ad['total']
    document_on_ad['clicked_percent_normalized'] = (document_on_ad['clicked'] + 12 * document_on_ad['clicked_percent'].mean()) / (12 + document_on_ad['total'])
    document_on_ad['likelihood_normalized'] = document_on_ad['clicked_percent_normalized'] / document_on_ad['uni_chance']
    document_on_ad['like_mul_total_normalized'] = document_on_ad['likelihood_normalized'] * document_on_ad['total']
    document_on_ad = document_on_ad.drop(document_on_ad.columns[[2,4,5,6,7,8]],axis = 1).groupby(['document_id','ad_id']).sum().reset_index()
    document_on_ad['score'] = document_on_ad['like_mul_total_normalized'] / document_on_ad['total']
    document_on_ad.drop(document_on_ad.columns[[2,3]],axis = 1, inplace=True)
    document_on_ad.sort_values('score',inplace=True, ascending=False)
    ad_strength = train_temp.groupby(['ad_id','ads_on_doc']).clicked.agg({'clicked' : 'sum', 'total' : 'count'}).reset_index()
    ad_strength['uni_chance'] = 1 / ad_strength['ads_on_doc']
    ad_strength['clicked_percent'] = ad_strength['clicked'] / ad_strength['total']
    ad_strength['clicked_percent_normalized'] = (ad_strength['clicked'] + 12 * ad_strength['clicked_percent'].mean()) / (12 + ad_strength['total'])
    ad_strength['likelihood_normalized'] = ad_strength['clicked_percent_normalized'] / ad_strength['uni_chance']
    ad_strength['like_mul_total_normalized'] = ad_strength['likelihood_normalized'] * ad_strength['total']
    ad_strength = ad_strength.drop(ad_strength.columns[[1,3,4,5,6,7]],axis = 1).groupby(['ad_id']).sum().reset_index()
    ad_strength['score'] = ad_strength['like_mul_total_normalized'] / ad_strength['total']
    ad_strength.drop(ad_strength.columns[[1,2]],axis = 1, inplace=True)
    ad_strength.sort_values('score',inplace=True, ascending=False)
    del train_temp
    return document_on_ad, ad_strength

In [3]:
#add document to clicks_train
events.drop(events.columns[[1,3,4,5]],axis = 1,inplace=True)
clicks_train = clicks_train.merge(events, how='left', on='display_id')
del events

In [ ]:
#1 - choose either 1 or 2
#split to validation and train, choosing display_id by random
display_ids = clicks_train.groupby(['display_id'])['display_id'].agg({'count' : 'count'}).reset_index().drop('count',axis = 1)
validation_displays = display_ids.sample(frac = 0.1)
clicks_train.reset_index(level=0, inplace=True)
validation = validation_displays.merge(clicks_train, how = 'inner', on = 'display_id')
train = clicks_train.drop(validation['index'])
del clicks_train

In [4]:
#2
#10% of full train for train, 2.5% validation, change (frac = 0.125) for different precent (4/5 train 1/5 validation)
#split to validation and train, choosing display_id by random
display_ids = clicks_train.groupby(['display_id'])['display_id'].agg({'count' : 'count'}).reset_index().drop('count',axis = 1)
chosen_displays = display_ids.sample(frac = 0.05)
train_displays , validation_displays = sms.train_test_split(chosen_displays, test_size = 0.2)
validation = validation_displays.merge(clicks_train, how = 'inner', on = 'display_id')
train = train_displays.merge(clicks_train, how = 'inner', on = 'display_id')
print train.shape
print validation.shape
del clicks_train, train_displays, validation_displays, display_ids, chosen_displays

(3484156, 4)
(871686, 4)


In [5]:
#proccess of one-hot on topics and categories
#remove all confidence levels below 0.3, can decide on another criteria later on
documents_categories = documents_categories[documents_categories.confidence_level > 0.3]
documents_topics = documents_topics[documents_topics.confidence_level > 0.3]
#drop confidence level
documents_categories.drop('confidence_level',axis=1, inplace=True)
documents_topics.drop('confidence_level',axis=1, inplace=True)

In [6]:
#prepare one-hot
topic_dummies = p.get_dummies(documents_topics['topic_id'],prefix="_ti")
categories_dummies = p.get_dummies(documents_categories['category_id'],prefix="_ci")
topics = documents_topics.drop('topic_id',1).join(topic_dummies)
topics = topics.groupby(['document_id']).sum().reset_index()
categories = documents_categories.drop('category_id',1).join(categories_dummies)
categories = categories.groupby(['document_id']).sum().reset_index()
del documents_categories, categories_dummies, topic_dummies , documents_topics

In [7]:
print promoted_content.head()
print categories.head()
print topics.head()

   ad_id  document_id  campaign_id  advertiser_id
0      1         6614            1              7
1      2       471467            2              7
2      3         7692            3              7
3      4       471471            2              7
4      5       471472            2              7
   document_id  _ci_1000  _ci_1100  _ci_1200  _ci_1202  _ci_1203  _ci_1204  \
0            1       0.0       0.0       0.0       0.0       0.0       0.0   
1            2       0.0       0.0       0.0       0.0       0.0       0.0   
2            3       0.0       0.0       0.0       0.0       0.0       0.0   
3            4       0.0       0.0       0.0       0.0       0.0       0.0   
4            5       0.0       0.0       0.0       0.0       0.0       0.0   

   _ci_1205  _ci_1206  _ci_1207    ...     _ci_1913  _ci_1914  _ci_1915  \
0       0.0       0.0       0.0    ...          0.0       0.0       0.0   
1       0.0       0.0       0.0    ...          0.0       0.0       0.0   
2     

In [8]:
#create promoted, promoted content (ad information) with its' categories and topics
#remove advertiser_id and campaign id
promoted_content.drop(promoted_content.columns[[2,3]],axis = 1,inplace=True)
promoted = promoted_content.merge(categories, how='left', on='document_id')
promoted = promoted.merge(topics, how='left', on='document_id').drop('document_id', axis=1)
del promoted_content
#all Nulls are of documents with no topic or category, equals to 0
promoted.fillna(0.0, inplace=True)

In [9]:
train_adoc, train_str = ad_on_doc_str(train)

In [10]:
#merge them into train and validation
train = train.merge(train_adoc, how='left', on = ['document_id', 'ad_id'])
train = train.merge(train_str, how='left', on = 'ad_id', suffixes= ('_adoc','_str'))
validation = validation.merge(train_adoc, how = 'left', on = ['document_id', 'ad_id'])
validation = validation.merge(train_str, how = 'left', on = 'ad_id', suffixes=('_adoc','_str'))
del train_adoc, train_str

In [11]:
#check ammount of missing data and fill
print validation.shape
doc_ad_null = validation.score_adoc.isnull().sum()
str_null = validation.score_str.isnull().sum()
print 'score_adoc nulls: ' + str(doc_ad_null)
print 'portion: ' + repr(float(doc_ad_null) / validation.shape[0])
print 'score_str nulls: ' + str(str_null)
print 'portion: ' + repr(float(str_null) / validation.shape[0])

(871686, 6)
score_adoc nulls: 529542
portion: 0.6074916885208664
score_str nulls: 19123
portion: 0.02193794554461125


In [12]:
#consider adding bit signaling the NAed ad_on_doc
#check if median or mean are better (or min?)

In [13]:
#for filling NAs, check median mean and min
print 'score_adoc mean = ' + repr(validation.score_adoc.mean())
print 'score_adoc median = ' + repr(validation.score_adoc.median())
print 'score_str mean = ' + repr(validation.score_str.mean())
print 'score_str median = ' + repr(validation.score_str.median())

score_adoc mean = 1.1258786409910577
score_adoc median = 1.037802584372659
score_str mean = 1.0209021240995964
score_str median = 0.9156235753390366


In [14]:
#fill NAs, pick with which method
validation.score_adoc = validation.score_adoc.fillna(validation.score_adoc.median())
validation.score_str = validation.score_str.fillna(validation.score_str.median())

In [15]:
#get predictors, dummy chunk just for label names, good for both 'not in chunks' and chunks
chunk = train[:10].merge(promoted,how = 'left',on = 'ad_id')
chunk = chunk.merge(categories, how='left', on='document_id',suffixes=('_ad', '_doc'))
chunk = chunk.merge(topics, how='left', on='document_id',suffixes=('_ad', '_doc'))
predictors=[x for x in train.columns if x not in ['index','display_id','ad_id','document_id','clicked']]

In [ ]:
#1 - choose either 1 or 2
#not in chunks
#build classifier
train = train.merge(categories, how = 'left', on = 'document_id')
train = train.merge(topics, how = 'left', on = 'document_id')
train = train.merge(promoted, how = 'left', on = 'ad_id')

In [ ]:
#not in chunks
#initiallize algorithm and fit
alg = GradientBoostingClassifier(n_estimators = 500, max_depth=4, min_samples_split = 4, max_leaf_nodes = 4)
#alg = RandomForestClassifier(random_state=1, n_estimators=3, min_samples_split=4, min_samples_leaf=2)
alg.fit(train[predictors], train["clicked"])

In [ ]:
#not in chunks
#run on validation
validation = validation.merge(categories, how = 'left', on = 'document_id')
validation = validation.merge(topics, how = 'left', on = 'document_id')
validation = validation.merge(promoted, how = 'left', on = 'ad_id')
predY=[]
predY = list(alg.predict_proba(validation[predictors]).astype(float)[:,1])

In [16]:
#2
#chunks - number of iterations for training
print train.shape[0]/10**5

34


In [ ]:
#chunks
#initiallize algorithm then fit it on chunks of size 10**5
i = 0
k = 3
#alg = GradientBoostingClassifier(n_estimators = 10, max_depth=4, min_samples_split = 4, max_leaf_nodes = 4, warm_start=True)
alg = RandomForestClassifier(random_state=1, n_estimators=3, min_samples_split=4, min_samples_leaf=2, warm_start=True)
for chunk in np.array_split(train, train.shape[0]/10**5):
    alg.n_estimators += k
    chunk = chunk.merge(promoted,how = 'left',on = 'ad_id')
    chunk = chunk.merge(categories, how='left', on='document_id',suffixes=('_ad', '_doc'))
    chunk = chunk.merge(topics, how='left', on='document_id',suffixes=('_ad', '_doc'))
    chunk.fillna(0.0, inplace=True)
    alg.fit(chunk[predictors], chunk["clicked"],)
    i += 1
    print i

1
2
3
4
5
6
7


In [ ]:
#save outputted tree with name [tree_name]
tree_name = 'fourthTree_n_est_drop_no_topics'
with open('/Users/astrachan/Desktop/outbrain/' + tree_name, 'wb') as f:
    cPickle.dump(alg, f)

In [ ]:
#chunks - number of iterations for validating
print (validation.shape[0]/10**5)

In [ ]:
#chunks - run outputted tree on chunks of size 10**5 of validation
predY=[]
i = 0
for chunk in np.array_split(validation[:10**6], validation[:10**6].shape[0]/10**5):
    chunk = chunk.merge(promoted,how = 'left',on = 'ad_id')
    chunk = chunk.merge(categories, how='left', on='document_id',suffixes=('_ad', '_doc'))
    chunk = chunk.merge(topics, how='left', on='document_id',suffixes=('_ad', '_doc'))
    chunk.fillna(0.0, inplace=True)
    chunk_pred=list(alg.predict_proba(chunk[predictors]).astype(float)[:,1])
    predY += chunk_pred
    i += 1
    print i
predY = list(alg.predict_proba(validation[predictors]).astype(float)[:,1])

In [37]:
#check score
predict = np.asarray(predY)
val_copy = validation.copy()
val_copy['predict'] = predict
#delete predict? validation? chunk? chunk_pred? predictors? predY? bentaim lo

In [38]:
#portion of correct display_ids predictions
max_rows = val_copy.groupby(['display_id'])['predict'].transform(max) == val_copy['predict']
final = val_copy[max_rows]
success = final[final['clicked'] == True]
float(len(success)) / float(len(final))

0.6920424300232756

In [39]:
#function evaluating how far were you from predicting right
val_copy.sort_values(['display_id', 'predict'], inplace=True, ascending=[True, False] )
val_copy["seq"] = np.arange(val_copy.shape[0])
Y_seq = val_copy[val_copy.clicked == 1].seq.values
Y_first = val_copy[['display_id', 'seq']].drop_duplicates(subset='display_id', keep='first').seq.values
Y_ranks = Y_seq - Y_first
score = np.mean(1.0 / (1.0 + Y_ranks))
print("MAP: %.12f" % score)

MAP: 0.816083156129
